# Implementing an SMC Likelihood Anealing Sampler

This notebook describes how to set up a SMC sampler from Algorithm 1. This SMC sampler assumes that the user will provide the set of temperatures used in likelihood annealing.

$$
\pi_t(\theta|y) \propto f(y|\theta)^{\gamma_t}\pi(\theta) \quad \text{for } t=1,\ldots,T,
$$

The user will specify the values $\gamma = (0, ..., 1)$ that define the sequence, the likelihood $f(y|\theta)$, prior $\pi$. 

## The Algorithm

Input: $\gamma = (\gamma_1=0,\gamma_2,...,\gamma_T=1)$ and random walk covariance $\Sigma$, number of MCMC move steps $R$.

1. Sample initial $\xi^i_1\sim \pi_1(\theta)$ and set $W_1^i = \frac{1}{N}$ for $i\in\{1,...,N\}$
2. For each time t = 2, ..., T

    (i) Calculate weights (Importance sampling)
            $$\tilde{w}_{t-1}^i = W_{t-1}^if(y|\theta = \xi^i_{t-1})^{\gamma_t - \gamma_{t-1}} \qquad W_{t-1}^i = \frac{\tilde{w}_{t-1}^i}{\sum_{j=1}^N\tilde{w}_{t-1}^j},
            $$
            for $i=1,\ldots,N.$

    (ii)  Resample $A_t^i$ ~ Cat $(W_{t-1}^1,...,W_{t-1}^N)$ for $i\in \{1,...,N\}$
    
    (iii) Diversify (MCMC) with $R$ RW proposals $\xi^* \sim \mathcal{N}(\xi, \Sigma)$
            $$
            \xi_{t}^i \sim K_{\pi_t}(\cdot \mid \theta = \xi_{t-1}^{A_{t-1}^i}) , \quad \text{  for  } i=1,\ldots,N.
            $$

### Coding utilities 

For numerical stability calculations will be done on the log scale - i.e. the user will specify the log likelihood and log prior. We also need the following function which will perform a numerically stable log sum of exponential numbers. We also need some necessary packages.


In [ ]:
function logsumexp(x)
    my_max = maximum(x)
    x_new = x .- my_max
    return my_max + log(sum(exp.(x_new)))
end

using Distributions, StatsBase, LinearAlgebra

### Coding the algorithm

The user will specify a function to simulate from the prior, evaluate the log prior, log likelihood, the choice of $\gamma$, $\Sigma$ and $R$.

In [ ]:
function smc_sampler_lanneal(N, simprior, logprior, loglike; γ = range(0, stop=1, length=15).^5, Σ = nothing, R=20)

    # Sample initial from prior and set weights
    ξ = simprior(N)
    log_W = ones(N) ./ N

    d = size(ξ, 2) # Get dimension of θ

    # If RW sigma is not specified use a default
    if isnothing(Σ)
        Σ = 2.38/sqrt(d) .* diagm(ones(d))
    end

    loglike_curr = fill(NaN, N)
    logprior_curr = fill(NaN, N)
    for i in 1:N
        loglike_curr[i] = loglike(ξ[i, :])
        logprior_curr[i] = logprior(ξ[i, :])
    end
  
    for t in 2:length(γ)

        # Calcuate (log) weights w and normalised weights W
        log_w = log_W .+ (γ[t] - γ[t-1]) .* loglike_curr
        log_W .= log_w .- logsumexp(log_w) 
        
        # Resample the particles
        inds = sample(1:N, Weights(exp.(log_W)), N) # Multinomial

        ξ = ξ[inds, :]
        loglike_curr = loglike_curr[inds]
        logprior_curr = logprior_curr[inds]
    
        # MCMC Diversify step
        for i in 1:N
            for k in 1:R
                ξ_prop = rand(MvNormal(ξ[i, :], Σ))
                loglike_prop = loglike(ξ_prop)
                logprior_prop = logprior(ξ_prop)
                log_rmh = γ[t] * (loglike_prop - loglike_curr[i]) + logprior_prop - logprior_curr[i]

                # Accept the proposal with probability RMH
                if rand() < exp(log_rmh)
                    ξ[i, :] = ξ_prop
                    loglike_curr[i] = loglike_prop
                    logprior_curr[i] = logprior_prop
                end
            end
        end
        println("Current γ = ",γ[t])
        println("Unique particles: ",length(unique(samples_curr[:, 1])))
    end
    return ξ
end

**NOTE: this code is also replicated in the smc_sampler_lanneal.jl**

## Exercises

Some exercises for solidifying knowledge:

1. Read and implement this SMC sampler for the predator-prey model: **lv_smc_example.ipynb**
2. Adjust the code to implement the adaptive version of the sampler (smc_sampler_lanneal)
3. Try running the SMC sampler using the package SequentialMonteCarlo.jl **lvsmc**